# Usage of Processors with Event Objects


The following example demonstrates the delivery of events to the opensearch output connector

In [4]:
import json
import uuid
from logprep.factory import Factory
from logprep.util.time import TimeParser
from logprep.event.log_event import LogEvent
from logprep.event.event_state import EventStateType
import logging
import sys

# Configure logging
logging.basicConfig(level=logging.DEBUG, stream=sys.stdout)

document = {
    "id": f"{uuid.uuid4()}",
    "@timestamp": str(TimeParser.now()),
    "user": {
        "name": "Hubert K. Kabal",
        "email": "kabal@example.com",
        "id": 12345,
    },
}

event = LogEvent(document, original=b"", state=EventStateType.RECEIVED)

print(f"Event before processing: {json.dumps(event.data, indent=2)}")

# adding a custom field to the event
config = {
    "my generic adder": {
        "type": "ng_generic_adder",
        "rules": [{"filter": "*", "generic_adder": {"add": {"event.tags": "generic added tag"}}}],
    }
}
processor = Factory.create(config)
processor.setup()
processor.process(event)
print(f"Event after processing: {json.dumps(event.data, indent=2)}")

Event before processing: {
  "id": "92411899-0f70-42e8-998b-79666244aa27",
  "@timestamp": "2025-08-22 07:55:51.662772+00:00",
  "user": {
    "name": "Hubert K. Kabal",
    "email": "kabal@example.com",
    "id": 12345
  }
}
DEBUG:Processor:GenericAdder (my generic adder) loaded 1 rules
DEBUG:Component:Checking health of my generic adder
DEBUG:Processor:GenericAdder (my generic adder) processing event LogEvent(data={'id': '92411899-0f70-42e8-998b-79666244aa27', '@timestamp': '2025-08-22 07:55:51.662772+00:00', 'user': {'name': 'Hubert K. Kabal', 'email': 'kabal@example.com', 'id': 12345}}, state=received)
Event after processing: {
  "id": "92411899-0f70-42e8-998b-79666244aa27",
  "@timestamp": "2025-08-22 07:55:51.662772+00:00",
  "user": {
    "name": "Hubert K. Kabal",
    "email": "kabal@example.com",
    "id": 12345
  },
  "event": {
    "tags": "generic added tag"
  }
}


In [5]:
print(f"Event before processing: {json.dumps(event.data, indent=2)}")

# Predetection example
config = {
    "almighty pre_detector": {
        "type": "ng_pre_detector",
        "outputs": [
            {"opensearch": "pseudonyms"}
        ],
        "rules": [
            {
                "filter": 'user.id: 12345',
                "pre_detector": {
                    "case_condition": "directly",
                    "id": "RULE_ONE_ID",
                    "mitre": [
                        "attack.something1",
                        "attack.something2"
                    ],
                    "severity": "critical",
                    "title": "Rule one",
                    "description": "Some malicious event."
                }
            }
        ],
    }
}
processor = Factory.create(config)
processor.setup()
processor.process(event)
print(f"Event after processing: {json.dumps(event.data, indent=2)}")
print(f"{len(event.extra_data)=}")
print(f"Event extra data: {json.dumps(event.extra_data[0].data, indent=2)}")

Event before processing: {
  "id": "92411899-0f70-42e8-998b-79666244aa27",
  "@timestamp": "2025-08-22 07:55:51.662772+00:00",
  "user": {
    "name": "Hubert K. Kabal",
    "email": "kabal@example.com",
    "id": 12345
  },
  "event": {
    "tags": "generic added tag"
  }
}
DEBUG:Processor:PreDetector (almighty pre_detector) loaded 1 rules
DEBUG:Component:Checking health of almighty pre_detector
DEBUG:Processor:PreDetector (almighty pre_detector) processing event LogEvent(data={'id': '92411899-0f70-42e8-998b-79666244aa27', '@timestamp': '2025-08-22 07:55:51.662772+00:00', 'user': {'name': 'Hubert K. Kabal', 'email': 'kabal@example.com', 'id': 12345}, 'event': {'tags': 'generic added tag'}}, state=received)
Event after processing: {
  "id": "92411899-0f70-42e8-998b-79666244aa27",
  "@timestamp": "2025-08-22 07:55:51.662772+00:00",
  "user": {
    "name": "Hubert K. Kabal",
    "email": "kabal@example.com",
    "id": 12345
  },
  "event": {
    "tags": "generic added tag"
  },
  "pre_de

In [6]:
print(f"Event before processing: {json.dumps(event.data, indent=2)}")

# Pseudonymization
config = {
    "almighty pseudonymizer": {
        "type": "ng_pseudonymizer",
        "pubkey_analyst": "../../../../../examples/exampledata/rules/pseudonymizer/example_analyst_pub.pem",
        "pubkey_depseudo": "../../../../../examples/exampledata/rules/pseudonymizer/example_depseudo_pub.pem",
        "regex_mapping": "../../../../../examples/exampledata/rules/pseudonymizer/regex_mapping.yml",
        "hash_salt": "a_secret_tasty_ingredient",
        "outputs": [
            {"opensearch": "pseudonyms"}
        ],
        "rules": [
            {
                "filter": "*",
                "pseudonymizer": {
                    "mapping": {
                        "user.name": "RE_WHOLE_FIELD",
                    }
                }
            }
        ],
        "max_cached_pseudonyms": 1000000
    }
}
processor = Factory.create(config)
processor.setup()
processor.process(event)
print(f"Event after processing: {json.dumps(event.data, indent=2)}")
print(f"{len(event.extra_data)=}")
print(f"Event extra data: {json.dumps(event.extra_data[1].data, indent=2)}")

Event before processing: {
  "id": "92411899-0f70-42e8-998b-79666244aa27",
  "@timestamp": "2025-08-22 07:55:51.662772+00:00",
  "user": {
    "name": "Hubert K. Kabal",
    "email": "kabal@example.com",
    "id": 12345
  },
  "event": {
    "tags": "generic added tag"
  },
  "pre_detection_id": "c8faf4cb-cdb4-4378-ad0b-9ac5f19fe75f"
}
DEBUG:Processor:Pseudonymizer (almighty pseudonymizer) loaded 1 rules
DEBUG:Component:Checking health of almighty pseudonymizer
DEBUG:Processor:Pseudonymizer (almighty pseudonymizer) processing event LogEvent(data={'id': '92411899-0f70-42e8-998b-79666244aa27', '@timestamp': '2025-08-22 07:55:51.662772+00:00', 'user': {'name': 'Hubert K. Kabal', 'email': 'kabal@example.com', 'id': 12345}, 'event': {'tags': 'generic added tag'}, 'pre_detection_id': 'c8faf4cb-cdb4-4378-ad0b-9ac5f19fe75f'}, state=received)
Event after processing: {
  "id": "92411899-0f70-42e8-998b-79666244aa27",
  "@timestamp": "2025-08-22 07:55:51.662772+00:00",
  "user": {
    "name": "<ps